In [2]:
from qiskit import Aer, QuantumCircuit, QuantumRegister, transpile, execute, IBMQ
import numpy as np
from qiskit.circuit import Qubit
from qiskit.providers.aer.backends.aerbackend import AerBackend
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram
from scipy import linalg

In [3]:
simulator = Aer.get_backend('qasm_simulator')

In [4]:
# Full Adder
# Register: 0 - x, 1 - y, 2 - carry, 3 - out
def simple_full_adder() -> QuantumCircuit:
     full_adder = QuantumCircuit(4, name='full_adder')
     full_adder.ccx(0, 1, 3)
     full_adder.cx(0, 1)
     full_adder.ccx(1, 2, 3)
     full_adder.cx(1, 2)
     full_adder.cx(0, 1)
     full_adder.swap(2, 3)
     return full_adder

full_adder_gate = simple_full_adder().to_gate()
full_adder_instruction = simple_full_adder().to_instruction()

In [5]:
def full_adder_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    if length == 1:
        return simple_full_adder()

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')
    carry = QuantumRegister(1, 'carry')

    adder = QuantumCircuit(x, y, out, carry, name=format("add_%s", str(length)))

    for i in range(0, length):
        adder.append(full_adder_instruction, [x[i], y[i], out[i], carry[0]])

    return adder

In [6]:
def xor_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')

    xor = QuantumCircuit(x, y, out, name=format("xor_%s", str(length)))

    for i in range(0, length):
        xor.cx(x[i], out[i])
        xor.cx(y[i], out[i])

    return xor

In [7]:
def vector_to_indices(v: int) -> list:
    ind = []
    for i in range(8):
        if (v >> i) & 0x1 == 0x1:
            ind.append(i)
    return ind


def s_box_circuit(anf: list, box_name: str) -> QuantumCircuit:
    assert len(anf) == (1 << 8)

    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    circuit = QuantumCircuit(x, out, name="s_box_" + box_name)

    for i in range(1 << 8):
        if anf[i] != 0:
            if i == 0:
                for ind in vector_to_indices(anf[i]):
                    circuit.x(out[ind])
            else:
                for ind in vector_to_indices(anf[i]):
                    circuit.mcx(x[vector_to_indices(i)], out[ind])

    return circuit


In [8]:
def add_f8_circuit() -> QuantumCircuit:
    return xor_circuit(8)

In [9]:
# Reduction polynomial x^8 + x^4 + x^3 + x^2 + 1

# Reduction matrix

Q = np.array([
    [1, 0, 1, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]
])

In [10]:
print(Q.transpose())


[[1 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 1 0]
 [1 0 1 0 1 1 0 0]
 [1 1 0 1 1 0 0 0]
 [1 1 1 0 0 0 1 0]
 [0 1 1 1 0 0 0 0]
 [0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 0]]


In [11]:
# C-NOT Synthesis

Qt = Q.copy().transpose()

In [12]:
Qt[2] = Qt[2] ^ Qt[0]
Qt[3] = Qt[3] ^ Qt[0]
Qt[4] = Qt[4] ^ Qt[0]

In [13]:
Qt[3] = Qt[3] ^ Qt[1]
Qt[4] = Qt[4] ^ Qt[1]
Qt[5] = Qt[5] ^ Qt[1]

In [14]:
Qt[4] = Qt[4] ^ Qt[2]
Qt[5] = Qt[5] ^ Qt[2]
Qt[6] = Qt[6] ^ Qt[2]

In [15]:
Qt[5] = Qt[5] ^ Qt[3]
Qt[6] = Qt[6] ^ Qt[3]
Qt[7] = Qt[7] ^ Qt[3]

In [16]:
Qt[6] = Qt[6] ^ Qt[4]
Qt[7] = Qt[7] ^ Qt[4]

In [17]:
Qt[7] = Qt[7] ^ Qt[5]

In [18]:
print(Qt)

[[1 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 1 0]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [19]:
Qtt = np.array(Qt).transpose()

In [20]:
print(Qtt)

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [1 0 0 0 1 0 0 0]
 [1 1 0 0 0 1 0 0]
 [1 1 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [21]:
Qtt[4] = Qtt[4] ^ Qtt[0]
Qtt[5] = Qtt[5] ^ Qtt[0]
Qtt[6] = Qtt[6] ^ Qtt[0]

In [22]:
Qtt[5] = Qtt[5] ^ Qtt[1]
Qtt[6] = Qtt[6] ^ Qtt[1]

In [23]:
Qtt[6] = Qtt[6] ^ Qtt[2]


In [24]:
print(Qtt)

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [25]:
def qt_reduction_circuit() -> QuantumCircuit:
    qt = QuantumCircuit(8, name="qt_reduction")

    qt.cx(4, 0)
    qt.cx(5, 0)
    qt.cx(6, 0)
    qt.cx(5, 1)
    qt.cx(6, 1)
    qt.cx(6, 2)

    qt.barrier()

    qt.cx(5, 7)
    qt.cx(4, 7)
    qt.cx(4, 6)
    qt.cx(3, 7)
    qt.cx(3, 6)
    qt.cx(3, 5)
    qt.cx(2, 6)
    qt.cx(2, 5)
    qt.cx(2, 4)
    qt.cx(1, 5)
    qt.cx(1, 4)
    qt.cx(1, 3)
    qt.cx(0, 4)
    qt.cx(0, 3)
    qt.cx(0, 2)

    return qt

qt_reduction_circuit().draw()

┌───┐┌───┐┌───┐                ░                                         »
q_0: ┤ X ├┤ X ├┤ X ├────────────────░─────────────────────────────────────────»
     └─┬─┘└─┬─┘└─┬─┘┌───┐┌───┐      ░                                         »
q_1: ──┼────┼────┼──┤ X ├┤ X ├──────░─────────────────────────────────────────»
       │    │    │  └─┬─┘└─┬─┘┌───┐ ░                                         »
q_2: ──┼────┼────┼────┼────┼──┤ X ├─░─────────────────────────────────■────■──»
       │    │    │    │    │  └─┬─┘ ░                                 │    │  »
q_3: ──┼────┼────┼────┼────┼────┼───░──────────────────■────■────■────┼────┼──»
       │    │    │    │    │    │   ░                  │    │    │    │    │  »
q_4: ──■────┼────┼────┼────┼────┼───░────────■────■────┼────┼────┼────┼────┼──»
            │    │    │    │    │   ░        │    │    │    │  ┌─┴─┐  │  ┌─┴─┐»
q_5: ───────■────┼────■────┼────┼───░───■────┼────┼────┼────┼──┤ X ├──┼──┤ X ├»
                 │         │    │   ░   │    │  ┌─┴─┐  │  ┌─┴─┐└───┘┌─┴─┐└───┘»
q_6: ────────────■─────────■────■───░───┼────┼──┤ X ├──┼──┤ X ├─────┤ X ├─────»
                                    ░ ┌─┴─┐┌─┴─┐└───┘┌─┴─┐└───┘     └───┘     »
q_7: ───────────────────────────────░─┤ X ├┤ X ├─────┤ X ├────────────────────»
                                    ░ └───┘└───┘     └───┘                    »
«                                        
«q_0: ──────────────────────■────■────■──
«                           │    │    │  
«q_1: ───────■────■────■────┼────┼────┼──
«            │    │    │    │    │  ┌─┴─┐
«q_2: ──■────┼────┼────┼────┼────┼──┤ X ├
«       │    │    │  ┌─┴─┐  │  ┌─┴─┐└───┘
«q_3: ──┼────┼────┼──┤ X ├──┼──┤ X ├─────
«     ┌─┴─┐  │  ┌─┴─┐└───┘┌─┴─┐└───┘     
«q_4: ┤ X ├──┼──┤ X ├─────┤ X ├──────────
«     └───┘┌─┴─┐└───┘     └───┘          
«q_5: ─────┤ X ├─────────────────────────
«          └───┘                         
«q_6: ───────────────────────────────────
«                                        
«q_7: ───────────────────────────────────
«

In [ ]:
def explode(*registers) -> list:
    exploded = []
    for reg in registers:
        if isinstance(reg, QuantumRegister):
            for index in range(reg.size):
                exploded.append(reg[index])
        elif isinstance(reg, Qubit):
            exploded.append(reg)
        else:
            raise Exception("Unexpected quantum storage type")
    return exploded

In [52]:
def mult_f8_circuit() -> QuantumCircuit:
    x = QuantumRegister(8, 'x')
    y = QuantumRegister(8, 'y')
    out = QuantumRegister(8, 'out')

    mult = QuantumCircuit(x, y, out, name='mult_f8')

    for i in range(1, 8):
        for j in range(8 - i):
            mult.ccx(x[i + j], y[7 - j], out[i - 1])

    mult.append(qt_reduction_circuit(), qargs=explode(out))

    for i in range(8):
        for j in range(i + 1):
            mult.ccx(x[j], y[i - j], out[i])

    return mult

mult_f8_circuit().draw()

»
  x_0: ──────────────────────────────────────────────────────────────────────»
                                                                             »
  x_1: ──■───────────────────────────────────────────────────────────────────»
         │                                                                   »
  x_2: ──┼────■─────────■────────────────────────────────────────────────────»
         │    │         │                                                    »
  x_3: ──┼────┼────■────┼─────────■──────────────■───────────────────────────»
         │    │    │    │         │              │                           »
  x_4: ──┼────┼────┼────┼────■────┼─────────■────┼──────────────■────────────»
         │    │    │    │    │    │         │    │              │            »
  x_5: ──┼────┼────┼────┼────┼────┼────■────┼────┼─────────■────┼────────────»
         │    │    │    │    │    │    │    │    │         │    │            »
  x_6: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────■────┼────┼─────────■──»
         │    │    │    │    │    │    │    │    │    │    │    │         │  »
  x_7: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────┼──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_0: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_1: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────┼──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_2: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_3: ──┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼────┼────┼────■──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_4: ──┼────┼────┼────┼────■────┼────┼────┼────┼────┼────■────┼────┼────┼──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_5: ──┼────┼────■────┼────┼────┼────┼────■────┼────┼────┼────┼────┼────┼──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_6: ──┼────■────┼────┼────┼────■────┼────┼────┼────┼────┼────■────┼────┼──»
         │    │    │    │    │    │    │    │    │    │    │    │    │    │  »
  y_7: ──■────┼────┼────■────┼────┼────┼────┼────■────┼────┼────┼────┼────┼──»
       ┌─┴─┐┌─┴─┐┌─┴─┐  │  ┌─┴─┐  │  ┌─┴─┐  │    │  ┌─┴─┐  │    │  ┌─┴─┐  │  »
out_0: ┤ X ├┤ X ├┤ X ├──┼──┤ X ├──┼──┤ X ├──┼────┼──┤ X ├──┼────┼──┤ X ├──┼──»
       └───┘└───┘└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐  │  └───┘┌─┴─┐  │  └───┘┌─┴─┐»
out_1: ───────────────┤ X ├─────┤ X ├─────┤ X ├──┼───────┤ X ├──┼───────┤ X ├»
                      └───┘     └───┘     └───┘┌─┴─┐     └───┘┌─┴─┐     └───┘»
out_2: ────────────────────────────────────────┤ X ├──────────┤ X ├──────────»
                                               └───┘          └───┘          »
out_3: ──────────────────────────────────────────────────────────────────────»
                                                                             »
out_4: ──────────────────────────────────────────────────────────────────────»
                                                                             »
out_5: ──────────────────────────────────────────────────────────────────────»
                                                                             »
out_6: ──────────────────────────────────────────────────────────────────────»
                                                                             »
out_7: ──────────────────────────────────────────────────────────────────────»
                                                                             »
«                                                                             »
«  x_0: ──────────────────────────────────────────────────────────────────────»
«                                             

In [36]:
def add_f64_circuit() -> QuantumCircuit:
    return xor_circuit(64)

In [37]:
def mult_const_f8_circuit(constant: int, name: str) -> QuantumCircuit:
    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    const_vector = []
    for i in range(8):
        const_vector.append((constant >> i) & 0x1)

    mult = QuantumCircuit(x, out, name='mult_f8_' + name)

    for i in range(1, 8):
        for j in range(8 - i):
            if const_vector[i + j] == 0x1:
                mult.cx(x[7 - j], out[i - 1])

    mult.append(qt_reduction_circuit(), qargs=explode(out))

    for i in range(8):
        for j in range(i + 1):
            if const_vector[j] == 0x1:
                mult.cx(x[i - j], out[i])

    return mult

░                 ░            ░       ░  ░  ░  ░ »
  x_0: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
  x_1: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
  x_2: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
  x_3: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
  x_4: ─────────────────■───░─────────────────░────────────░───────░──░──░──░─»
                        │   ░                 ░            ░       ░  ░  ░  ░ »
  x_5: ────────────■────┼───░─────────────■───░────────────░───────░──░──░──░─»
                   │    │   ░             │   ░            ░       ░  ░  ░  ░ »
  x_6: ───────■────┼────┼───░────────■────┼───░────────■───░───────░──░──░──░─»
              │    │    │   ░        │    │   ░        │   ░       ░  ░  ░  ░ »
  x_7: ──■────┼────┼────┼───░───■────┼────┼───░───■────┼───░───■───░──░──░──░─»
       ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐ ░   │    │    │   ░   │    │   ░   │   ░  ░  ░  ░ »
out_0: ┤ X ├┤ X ├┤ X ├┤ X ├─░───┼────┼────┼───░───┼────┼───░───┼───░──░──░──░─»
       └───┘└───┘└───┘└───┘ ░ ┌─┴─┐┌─┴─┐┌─┴─┐ ░   │    │   ░   │   ░  ░  ░  ░ »
out_1: ─────────────────────░─┤ X ├┤ X ├┤ X ├─░───┼────┼───░───┼───░──░──░──░─»
                            ░ └───┘└───┘└───┘ ░ ┌─┴─┐┌─┴─┐ ░   │   ░  ░  ░  ░ »
out_2: ─────────────────────░─────────────────░─┤ X ├┤ X ├─░───┼───░──░──░──░─»
                            ░                 ░ └───┘└───┘ ░ ┌─┴─┐ ░  ░  ░  ░ »
out_3: ─────────────────────░─────────────────░────────────░─┤ X ├─░──░──░──░─»
                            ░                 ░            ░ └───┘ ░  ░  ░  ░ »
out_4: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
out_5: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
out_6: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
out_7: ─────────────────────░─────────────────░────────────░───────░──░──░──░─»
                            ░                 ░            ░       ░  ░  ░  ░ »
«                              ░            ░                 ░                »
«  x_0: ───────────────────■───░────────■───░─────────────■───░────────────────»
«                          │   ░        │   ░             │   ░                »
«  x_1: ───────────────────┼───░───■────┼───░────────■────┼───░─────────────■──»
«                          │   ░   │    │   ░        │    │   ░             │  »
«  x_2: ───────────────────┼───░───┼────┼───░───■────┼────┼───░────────■────┼──»
«                          │   ░   │    │   ░   │    │    │   ░        │    │  »
«  x_3: ───────────────────┼───░───┼────┼───░───┼────┼────┼───░───■────┼────┼──»
«                          │   ░   │    │   ░   │    │    │   ░   │    │    │  »
«  x_4: ───────────────────┼───░───┼────┼───░───┼────┼────┼───░───┼────┼────┼──»
«                          │   ░   │    │   ░   │    │    │   ░   │    │    │  »
«  x_5: ───────────────────┼───░───┼────┼───░───┼────┼────┼───░───┼────┼────┼──»
«                          │   ░   │    │   ░   │    │    │   ░   │    │    │  »
«  x_6: ───────────────────┼───░───┼────┼───░───┼────┼────┼───░───┼────┼────┼──»
«                          │   ░   │    │   ░   │    │    │   ░   │    │    │  »
«  x_7: ───────────────────┼───░───┼────┼───░───┼────┼────┼───░───┼────┼────┼──»
«       ┌───────────────┐┌─┴─┐ ░   │    │   ░   │    │    │   ░   │    │    │  »
«out_0: ┤0 

In [38]:
reduction_poly = 0b00011101
gf_2_8 = [0x1]

elem = 0x1
for _ in range(1, 1 << 8):
    elem = elem << 1
    if elem >> 8 == 0x1:
        elem = (elem ^ reduction_poly) & 0xFF
    gf_2_8.append(elem)

In [59]:
def inplace_add_f8():
    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    circuit = QuantumCircuit(x, out, name='inplace_add_f8')

    for i in range(8):
        circuit.cx(x[i], out[i])

    return circuit

In [61]:
def mult_alpha_f64_circuit() -> QuantumCircuit:
    add_f8 = inplace_add_f8().to_instruction()

    xs = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    outs = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]

    registers = []
    registers.extend(xs)
    registers.extend(outs)

    circuit = QuantumCircuit(*registers, name='mult_alpha_f64')

    circuit.append(mult_const_f8_circuit(gf_2_8[2  ], 'b^2'  ), qargs=explode(xs[7], outs[0]))
    circuit.append(mult_const_f8_circuit(gf_2_8[70 ], 'b^70' ), qargs=explode(xs[7], outs[3]))
    circuit.append(mult_const_f8_circuit(gf_2_8[224], 'b^224'), qargs=explode(xs[7], outs[4]))
    circuit.append(add_f8, qargs=explode(outs[0], outs[5]))
    circuit.append(mult_const_f8_circuit(gf_2_8[166], 'b^166'), qargs=explode(xs[7], outs[6]))

    for i in range(7):
        circuit.append(add_f8, qargs=explode(xs[i], outs[i + 1]))

    return circuit

»
  x_0_0: ───────■──────────────────────────────────────────────────────────────»
                │                                                              »
  x_0_1: ───────┼────■─────────────────────────────────────────────────────────»
                │    │                                                         »
  x_0_2: ───────┼────┼────■────────────────────────────────────────────────────»
                │    │    │                                                    »
  x_0_3: ───────┼────┼────┼────■───────────────────────────────────────────────»
                │    │    │    │                                               »
  x_0_4: ───────┼────┼────┼────┼────■──────────────────────────────────────────»
                │    │    │    │    │                                          »
  x_0_5: ───────┼────┼────┼────┼────┼────■─────────────────────────────────────»
                │    │    │    │    │    │                                     »
  x_0_6: ───────┼────┼────┼────┼────┼────┼────■────────────────────────────────»
                │    │    │    │    │    │    │                                »
  x_0_7: ───────┼────┼────┼────┼────┼────┼────┼────■───────────────────────────»
                │    │    │    │    │    │    │    │                           »
  x_1_0: ───────┼────┼────┼────┼────┼────┼────┼────┼────■──────────────────────»
                │    │    │    │    │    │    │    │    │                      »
  x_1_1: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────■─────────────────»
                │    │    │    │    │    │    │    │    │    │                 »
  x_1_2: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────────────»
                │    │    │    │    │    │    │    │    │    │    │            »
  x_1_3: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■───────»
                │    │    │    │    │    │    │    │    │    │    │    │       »
  x_1_4: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_1_5: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_1_6: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_1_7: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_0: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_1: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_2: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_3: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_4: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_5: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_6: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_2_7: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │    │    │    │    │    │    │    │    │    │    │  »
  x_3_0: ───────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
                │    │    │  

In [63]:
def mult_inv_alpha_f64_circuit() -> QuantumCircuit:
    add_f8 = inplace_add_f8().to_instruction()

    xs = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    outs = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]

    registers = []
    registers.extend(xs)
    registers.extend(outs)

    circuit = QuantumCircuit(*registers, name='mult_inv_alpha_f64')

    circuit.append(mult_const_f8_circuit(gf_2_8[68 ], 'b^68' ), qargs=explode(xs[0], outs[2]))
    circuit.append(mult_const_f8_circuit(gf_2_8[222], 'b^222'), qargs=explode(xs[0], outs[3]))
    circuit.append(add_f8, qargs=explode(xs[0], outs[4]))
    circuit.append(mult_const_f8_circuit(gf_2_8[164], 'b^164'), qargs=explode(xs[0], outs[5]))
    circuit.append(mult_const_f8_circuit(gf_2_8[168], 'b^168'), qargs=explode(xs[0], outs[6]))

    for i in range(7):
        circuit.append(add_f8, qargs=explode(xs[i + 1], outs[i]))

    return circuit